# Сбор данных с Web-scraping и API для социально-научных исследований
---
## Семинары 15-16. Selenium: продолжение. Менеджмент файлов на компьютере с помощью os.

---
*ФСН, ОП "Политология", 2023-2024 гг.*

Лика Капустина,

lkapustina@hse.ru

**План занятия:**

3. [3. Selenium: работа со значениями элементов и обработка html страницы (привет, BeautifulSoup)](#part3)
4. [4. Selenium: явные и неявные ожидания и обработка исключений](#part4)
5. [5. Selenium: ActionChains](#part5)
6. [6. os: менеджмент файлов на вашем компьютере](#part6)
7. [Дополнительно: что еще можно делать с помощью Selenium? Краткий обзор возможностей](#partlast)
---

На прошлом занятии мы с вами обсудили старт работы с Selenium – как создавать объект `webdriver`, как искать элементы на странице, как взаимодействовать с кнопками и полями. На этом занятии мы обсудим методы `WebElement`, работу с html-разметой страниц, явные и неявные ожидания, что такое ActionChains и как с помощью этого класса упростить себе работу, как скачивать файлы с сайтов и размещать их в нужных папках на вашем компьютере автоматизированно с помощью `os`. **Весь материал в этом ноутбуке не был представлен на семинаре 13-14**.

---

**Основные ссылки:**
- [Документация Selenium на Python](https://selenium-python.readthedocs.io);
- [Неофициальный перевод документации Selenium на Python на русский](https://habr.com/ru/articles/248559/);
- [Документация BeautifulSoup на русском](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html);
- [Документация os](https://docs.python.org/3/library/os.html);
- [Распространенные exceptions в Selenium](https://www.selenium.dev/selenium/docs/api/py/common/selenium.common.exceptions.html).

**Продолжим работать с Selenium. Начнем с подгрузки всех необходимых библиотек и классов:**

In [212]:
# импорт основных библиотек
import pandas as pd # работа с таблицами
import numpy as np  # математические операции и пр.
import tqdm         # для дальнейшего импорта прогресс-бара
import time         # для обычных пауз

from selenium import webdriver # импорт драйвера
from selenium.webdriver.chrome.service import Service # импортируем сущность Service
from webdriver_manager.chrome import ChromeDriverManager # импорт драйвера для Google Chrome
from selenium.webdriver.common.by import By # поиск по локаторам
from selenium.webdriver.common.keys import Keys # отправление ключей
from selenium.webdriver.chrome.options import Options 

In [406]:
# запуск вебдрайвера
options = Options() # создаем сущность Options
options.add_argument("start-maximized") # уточняем, что хотим чтобы при открытии окно было открыто максимально широко
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # создаем вебдрайвер

## 3. Selenium: работа со значениями элементов и обработка html страницы<a name='part3'></a>

В этом разделе мы обсудим работу со значениями элементов и вспомним работу со старым-добрым BeautifulSoup. Но сперва откроем страницу поиска Московского городского суда со следующими значениями:
* Инстанция – `первая`;
* Год – `2023`;
* В тексте встречается слово `НИУ ВШЭ`.

In [214]:
link = 'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=&letterNumber=&caseNumber=&participant=&codex=&judge=&publishingState=&documentType=&documentText=НИУ+ВШЭ&year=2023&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus='
wb.get(link) # открываем ссылку

### `WebElement`

Сущность, с которым мы будем работать с вами сейчас – `WebElement`, то есть, некоторая абстракция, которая стоит за конкретным элементом в разметке конкретной страницы. Давайте воспользуемся [изящным решением со stackoverflow](https://stackoverflow.com/questions/9058305/getting-attributes-of-a-class) и посмотрим на то, какие атрибуты есть у элементов. Часть из них мы уже видели (`find_element`, `find_elements`, `click`, `send_keys`), а с частью поработаем сейчас:

[Документация по классу WebElement доступна по ссылке](https://www.selenium.dev/selenium/docs/api/py/webdriver_remote/selenium.webdriver.remote.webelement.html).

In [215]:
from selenium.webdriver.remote.webelement import WebElement

def props(cls):   
    return [i for i in cls.__dict__.keys() if i[:1] != '_']

properties = props(WebElement)
properties # атрибуты класса WebElement

['tag_name',
 'text',
 'click',
 'submit',
 'clear',
 'get_property',
 'get_dom_attribute',
 'get_attribute',
 'is_selected',
 'is_enabled',
 'find_element_by_id',
 'find_elements_by_id',
 'find_element_by_name',
 'find_elements_by_name',
 'find_element_by_link_text',
 'find_elements_by_link_text',
 'find_element_by_partial_link_text',
 'find_elements_by_partial_link_text',
 'find_element_by_tag_name',
 'find_elements_by_tag_name',
 'find_element_by_xpath',
 'find_elements_by_xpath',
 'find_element_by_class_name',
 'find_elements_by_class_name',
 'find_element_by_css_selector',
 'find_elements_by_css_selector',
 'send_keys',
 'is_displayed',
 'location_once_scrolled_into_view',
 'size',
 'value_of_css_property',
 'location',
 'rect',
 'aria_role',
 'accessible_name',
 'screenshot_as_base64',
 'screenshot_as_png',
 'screenshot',
 'parent',
 'id',
 'find_element',
 'find_elements']

###  `element.text` и `element.get_attribute()`

В Selenium есть два метода, которые позволяют работать с значениями конкретных `webelement` – это `element.text` и `element.get_attribute()`. Разберем на примере:



**Как получить текст, который содержит определенный элемент?** С помощью атрибута `.text`:

**Найдем .CLASS_NAME, отвечающий за надпись `По вашему запросу найдено записей: 19`:**

In [216]:
wb.find_element(By.CLASS_NAME, 'resultsearch_text') # нашли элемент

<selenium.webdriver.remote.webelement.WebElement (session="06d3cad1f504d72f4f6d894c6f78a7de", element="f.136A8665D16BEC5DF58BE72F43659528.d.28BD3DE2A264DEFE11924AC230C94D7F.e.275")>

In [217]:
wb.find_element(By.CLASS_NAME, 'resultsearch_text').text # получили текст, который стоит за элементом

'По вашему запросу найдено записей: 19\nНажмите, чтобы развернуть таблицу'

In [221]:
wb.find_element(By.CLASS_NAME, 'resultsearch_text').text.split('\n')[0].split()[-1]

'19'

**На странице поиска в самом низу страницы отображается, сколько всего страниц с делами по такому запросу есть на сайте. Вопрос к вам – для чего нам может быть полезно это значение?**

Посмотрим на html, стоящий за нужным нам элементом `2`:

```
<input type="hidden" id="paginationFormMaxPages" value="2">
```

In [143]:
# попытаемся пойти простым путем и просто получить текст страницы
wb.find_element(By.ID, 'paginationFormMaxPages').text # ничего не получилось..

''

Если не получилось поработать с одним методом, нужно идти дальше. Смотрите, значение, которое нам нужно получить – это цифру `2` (число страниц с делами). Для этого посмотрим, для какого атрибута выставлено значение `2`.

In [224]:
wb.find_element(By.ID, 'paginationFormMaxPages').get_attribute('value') # и укажем в качестве аргумента метода

'2'

In [226]:
wb.find_element(By.TAG_NAME, 'strong').text

'Истец:'

In [229]:
for i in wb.find_elements(By.TAG_NAME, 'strong'):
    print(i.text)

Истец:
Ответчик:
Истец:
Ответчик:
Истец:
Ответчик:
Истец:
Ответчик:
Третья сторона со стороны истца:
Третья сторона со стороны ответчика:
Истец:
Ответчик:
Истец:
Ответчик:
Административный истец:
Административный ответчик:
Истец:
Ответчик:
Истец:
Ответчик:
Истец:
Ответчик:
Административный истец:
Административный ответчик:
Административный истец:
Административный ответчик:
Административный истец:
Административный ответчик:
Подсудимый:
Подсудимый:
































**Кроме этих двух методов есть и другие, которые позволяют получить информацию о конкретном элементе:**

In [232]:
wb.find_element(By.ID, 'paginationFormMaxPages').size # ширина и длина элемента

{'height': 0, 'width': 0}

In [233]:
wb.find_element(By.ID, 'paginationFormMaxPages').location # местонахождение элемента на странице

{'x': 144, 'y': 4014}

**Поздравляю! Вы освоили некоторые базовые методы работы с конкретными элементами**

<p></p>
<center><b><font size=4>Задача №1. Работа с элементами страницы</font></b></center>

Давайте закрепим успех. **Поработайте с элементами страницы и напечатайте на экран следующую информацию:**

* `Заголовок статьи: <значение>`
* `Поле "Найдите дело или задайте вопрос помощнику" принимает в качестве type значение: <значение>`
* `Высота кнопки "Противодействие коррупции": <значение>`
* `Ширина кнопки "Противодействие коррупции": <значение>`

In [329]:
link = 'https://mos-gorsud.ru/mgs/news/komanda-federalnyh-sudej-rossijskoj-federatsii-zanyala-1-mesto-po-itogam-xv-spartakiady-sredi-federalnyh-organov-gosudarstvennoj-vlasti-rossijskoj-federatsii'
wb.get(link)

**Смотрим на элемент заголовка:**

```
<h1>
                            Команда Федеральных судей Российской Федерации заняла 1 место по итогам XV Спартакиады среди федеральных органов государственной власти Российской Федерации
    
                                            </h1>
```

In [330]:
text_of_article = wb.find_element(By.TAG_NAME, 'h1').text # получаем текст первого элемента под тегом h1
print(f'Заголовок статьи: {text_of_article}')

Заголовок статьи: Команда Федеральных судей Российской Федерации заняла 1 место по итогам XV Спартакиады среди федеральных органов государственной власти Российской Федерации


**Смотрим на поле `Найдите дело или задайте вопрос помощнику`**

```
<input type="text" autocomplete="off" maxlength="255" placeholder="Введите атрибуты судебного дела или опишите ситуацию, с которой вам нужна помощь" class="search-input">
```

In [331]:
value = wb.find_element(By.CLASS_NAME, 'search-input').get_attribute('type') # внутри - название атрибута
print(f'Поле "Найдите дело или задайте вопрос помощнику" принимает в качестве type значение: {value}')

Поле "Найдите дело или задайте вопрос помощнику" принимает в качестве type значение: text


**Смотрим на поле `Противодействие коррупции`**

```
<img src="/resources/img/hand_corruption.png?v=3d3" width="45" alt="Противодействие коррупции">
```

In [332]:
button_corruption = wb.find_element(By.CSS_SELECTOR, 'body > div.wrapper > main > aside > div.sidebarslider-wrapper > div:nth-child(2) > div > div > div > a > div > span')
print(button_corruption.size)

{'height': 44, 'width': 169}


In [334]:
print(f"Высота кнопки: {button_corruption.size['height']}")

Высота кнопки: 44


In [335]:
print(f"Ширина кнопки: {button_corruption.size['width']}")

Ширина кнопки: 169


### Способы перехода на другие страницы

Теперь давайте обсудим, как вы можете переключаться между страницами. **Во-первых, вы можете найти кнопку и кликнуть на кнопку "следующая страница"**.

In [246]:
link = 'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=&letterNumber=&caseNumber=&participant=&codex=&judge=&publishingState=&documentType=&documentText=НИУ+ВШЭ&year=2023&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus='
wb.get(link) # открываем ссылку с поиском дел

In [248]:
wb.find_element(By.CLASS_NAME, 'next').click()

In [164]:
next_page_button = wb.find_element(By.CLASS_NAME, 'next') # кнопка "далее"
print(next_page_button)

<selenium.webdriver.remote.webelement.WebElement (session="0baf98523354fbd8737c0bac4739b491", element="f.8B67E3298034E2B8CF7E15063FEA6E36.d.C88775E6220A17733B1475B186C19654.e.1606")>


После того, как элемент найден, осталось только на него кликнуть.

In [165]:
time.sleep(3) # делаем паузу
next_page_button.click() # нажимаем на кнопку

При желании вы в цикле могли бы пройтись по всем страницам и каждый раз для перехода на следующую страницу нажимать на эту кнопку.

**Но есть ли более оптимальные опции?** Да. Для этого давайте взглянем на ссылку: ```https://mos-gorsud.ru/search??documentText=НИУ+ВШЭ&instance=1&year=2023&formType=fullForm&page=2```. 

Интересно, что если просто кликнуть на адресное поле и скопировать значение, вы получите отличающуюся строку: ```https://mos-gorsud.ru/search?documentText=%D0%9D%D0%98%D0%A3+%D0%92%D0%A8%D0%AD&instance=1&year=2023&formType=fullForm&page=2```.

Если воспользоваться методом `.current_url`, он вернет то же самое: ```https://mos-gorsud.ru/search?documentText=%D0%9D%D0%98%D0%A3+%D0%92%D0%A8%D0%AD&instance=1&year=2023&formType=fullForm&page=2'```.

Давайте сразу сгенерируем ссылки, которые в дальнейшем могли бы обойти в цикле и проверим, работают ли они. Для примера возьмем дело `212`:

In [250]:
pages_with_cases = \
[f'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page={i}' \
 for i in range(1, 47+1)] # поскольку на сайте всего 47 страниц с ссылками на дела по статье Ст. 212 УК РФ
pages_with_cases

['https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=1',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=2',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=3',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=4',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=5',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=6',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=7',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=8',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=9',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=10',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=11',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=12',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page=13',
 'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullFo

In [251]:
wb.get(pages_with_cases[0]) # откроем первую ссылку
time.sleep(3) # сделаем паузу
wb.get(pages_with_cases[-1]) # откроем последнюю ссылку

**Поздравляю! Вы научились генерировать ссылки или использовать альтернативные методы чтобы проходиться по однотипным страницам**.

### `wb.page_source`

**Перейдем к обсуждению работы с html страницы**. На этапе сбора данных работа с html страницы ничем не отличается от того, что мы с вами делали когда обсуждали `requests`.

Давайте соберем ссылки на все дела, которые выпали нам в поиске с параметрами `Первая инстанция`, `2023 год`, в тексте присутствует `НИУ ВШЭ`.

In [252]:
link = 'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=&letterNumber=&caseNumber=&participant=&codex=&judge=&publishingState=&documentType=&documentText=НИУ+ВШЭ&year=2023&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus='
wb.get(link) # открываем ссылку с поиском дел

In [253]:
html = wb.page_source # сохраним html страницы с помощью .page_source
soup = BeautifulSoup(html) # сварим суп

In [ ]:
soup

In [113]:
#soup # посмотрим на него

Давайте создадим список, в который соберем все ссылки на дела со страницы поиска. Как выглядит элемент с ссылкой на дело?

```<a target="_blank" class="detailsLink" href="/rs/basmannyj/services/cases/civil/details/7da80261-cd6f-11ed-91bc-678868b51ab8?year=2023&amp;formType=fullForm">02-3282/2023</a>```

Давайте воспользуемся атрибутом `class` и его значением - `detailsLink`. Далее мы обращаемся к нашему объекту `soup` и пользуемся старым добрым методом `.find_all()`, чтобы найти все элементы, удовлетворяющие деталям поиска:

*Если вы забыли как пользоваться методами BeautifulSoup, посмотрите материалы семинаров 3-6 или обратитесь к [документации BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html)*.

In [258]:
wb.current_url

'https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=&letterNumber=&caseNumber=&participant=&codex=&judge=&publishingState=&documentType=&documentText=%D0%9D%D0%98%D0%A3+%D0%92%D0%A8%D0%AD&year=2023&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus='

In [261]:
# как получить ссылку? Запросить атрибут href
soup.find('a', {'class':'detailsLink'})['href']

'/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm'

In [262]:
# альтернативный способ
soup.find('a', {'class':'detailsLink'}).get('href')

'/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm'

In [264]:
# чтобы получить значение атрибута, используем .get(атрибут)
soup.find('a', {'class':'detailsLink'}).get('target')

'_blank'

In [268]:
my_links = [f'https://mos-gorsud.ru{i.get("href")}' for i in soup.find_all('a', {'class':'detailsLink'})]
my_links = [i for i in my_links if i.find('details') != -1]
my_links # получили корректные ссылки

['https://mos-gorsud.ru/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/horoshevskij/services/cases/civil/details/63577af0-c944-11ed-ab7e-afa5fd1f1a80?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/7da80261-cd6f-11ed-91bc-678868b51ab8?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/tushinskij/services/cases/civil/details/48b8d670-be69-11ed-bcc6-6dacb38b2616?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/chertanovskij/services/cases/civil/details/90d28981-c281-11ed-b232-6fb0d5e1e88d?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/6afd9811-934e-11ed-aa38-dd42c3d462a2?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/tverskoj/services/cases/kas/details/9d1647c1-890c-11ee-bde3-e15247667f25?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/deta

Откроем первую ссылку

In [168]:
wb.get(links[0]) # все работает!

<p></p>
<center><b><font size=4>Задача №2. Сбор данных со страницы</font></b></center>

**Шаг 1. Вспомните работу с `BeautifulSoup` и напишите функцию `get_info_about_case(link)`, которая принимает на вход ссылку на страницу дела, и возвращает словарь (`dict`) с информацией из таблички в начале страницы и ссылки:**

Например, для первого дела вы получите следующий результат:
```
{'Уникальный идентификатор дела': '77RS0018-02-2023-004059-43',
 'Номер дела ~ материала': '02-4668/2023                                        ∼                                     М-2315/2023',
 'Стороны': 'Истец: Борсов А.И.Ответчик: ФГАОУ ВО "Национальный исследовательский университет "Высшая школа экономики"',
 'Дата поступления': '24.03.2023',
 'Дата рассмотрения дела в первой инстанции': '20.06.2023',
 'Дата вступления решения в силу': '14.12.2023',
 'Cудья': 'Юдина И.В.',
 'Категория дела': '179 - О защите прав потребителей - из договоров в сфере торговли, услуг, выполнения работ',
 'Номер дела в суде вышестоящей инстанции': '33-48031/2023',
 'Текущее состояние': 'Удовлетворено, 20.06.2023',
 'Решение апелляции': 'Отменено, 14.12.2023',
 'link': 'https://mos-gorsud.ru/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm'}
```

In [336]:
# YOUR CODE HERE
link = 'https://mos-gorsud.ru/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm'
wb.get(link) # открываем ссылку
html = wb.page_source # получаем html
soup = BeautifulSoup(html) # обрабатываем html в суп

Как найти элементы слева? Смотрим на примеры – их объединяет тег `div` и класс `left`:

```<div class="left">                        Уникальный идентификатор дела
        </div>```
        
```<div class="left">                Номер дела ~ материала
    </div>```

In [337]:
# значения из таблицы с левой стороны
columns = [i.text.strip() for i in soup.find_all('div', {'class':'left'})]
columns

['Уникальный идентификатор дела',
 'Номер дела ~ материала',
 'Стороны',
 'Дата поступления',
 'Дата рассмотрения дела в первой инстанции',
 'Дата вступления решения в силу',
 'Cудья',
 'Категория дела',
 'Номер дела в суде вышестоящей инстанции',
 'Текущее состояние',
 'Решение апелляции']

Как найти элементы справа? Смотрим на примеры – их объединяет тег `div` и класс `right`:

```
<div class="right">            77RS0018-02-2023-004059-43
        </div>
```
        
```
<div class="right">
                    02-4668/2023

            
                            ∼                                     М-2315/2023                                    
    </div>
```

In [338]:
# значения из таблицы с правой стороны
rows = [" ".join(i.text.strip().replace('\n', ' ').split()) for i in soup.find_all('div', {'class':'right'})]
rows

['77RS0018-02-2023-004059-43',
 '02-4668/2023 ∼ М-2315/2023',
 'Истец: Борсов А.И.Ответчик: ФГАОУ ВО "Национальный исследовательский университет "Высшая школа экономики"',
 '24.03.2023',
 '20.06.2023',
 '14.12.2023',
 'Юдина И.В.',
 '179 - О защите прав потребителей - из договоров в сфере торговли, услуг, выполнения работ',
 '33-48031/2023',
 'Удовлетворено, 20.06.2023',
 'Отменено, 14.12.2023']

In [340]:
# генерируем пары ключ-значение
my_dict = {columns[i]:rows[i] for i in range(len(columns))} # генератор словарей
my_dict

{'Уникальный идентификатор дела': '77RS0018-02-2023-004059-43',
 'Номер дела ~ материала': '02-4668/2023 ∼ М-2315/2023',
 'Стороны': 'Истец: Борсов А.И.Ответчик: ФГАОУ ВО "Национальный исследовательский университет "Высшая школа экономики"',
 'Дата поступления': '24.03.2023',
 'Дата рассмотрения дела в первой инстанции': '20.06.2023',
 'Дата вступления решения в силу': '14.12.2023',
 'Cудья': 'Юдина И.В.',
 'Категория дела': '179 - О защите прав потребителей - из договоров в сфере торговли, услуг, выполнения работ',
 'Номер дела в суде вышестоящей инстанции': '33-48031/2023',
 'Текущее состояние': 'Удовлетворено, 20.06.2023',
 'Решение апелляции': 'Отменено, 14.12.2023'}

In [ ]:
def get_info_about_case(link):
    wb.get(link) # открыли ссылку
    
    # обработали html-разметку
    html = wb.page_source
    soup = BeautifulSoup(html)
    columns = [i.text.strip() for i in soup.find_all('div', {'class':'left'})]
    rows = [" ".join(i.text.strip().replace('\n', ' ').split()) for i in soup.find_all('div', {'class':'right'})]
    # получаем словарь
    my_dict = {columns[i]:rows[i] for i in range(len(columns))}
    return my_dict

Как можно создать `pandas.DataFrame` из полученного словаря? С помощью функции `pd.json_normalize()`:

In [309]:
# ключи становятся названиями колонок,
# значения - значениями в соответствующих колонках
pd.json_normalize(my_dict)

,Уникальный идентификатор дела,Номер дела ~ материала,Стороны,Дата поступления,Дата рассмотрения дела в первой инстанции,Дата вступления решения в силу,Cудья,Категория дела,Номер дела в суде вышестоящей инстанции,Текущее состояние,Решение апелляции
0,77RS0018-02-2023-004059-43,02-4668/2023 ∼ М-2315/2023,"Истец: Борсов А.И.Ответчик: ФГАОУ ВО ""Национал...",24.03.2023,20.06.2023,14.12.2023,Юдина И.В.,179 - О защите прав потребителей - из договоро...,33-48031/2023,"Удовлетворено, 20.06.2023","Отменено, 14.12.2023"


**Шаг 2. Напишите функцию `get_cases_data(links)`, которая принимает на вход список с ссылками `links`, проходится по нему, собирает список из словарей с помощью функции `get_info_about_case()`, и возвращает `pandas.DataFrane` с информацией о судебных делах**.

*Подсказка:*

![Мем](https://pbs.twimg.com/media/E4P9gAtXwAMjiHu.jpg)

In [ ]:
links = ['https://mos-gorsud.ru/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/horoshevskij/services/cases/civil/details/63577af0-c944-11ed-ab7e-afa5fd1f1a80?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/7da80261-cd6f-11ed-91bc-678868b51ab8?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/tushinskij/services/cases/civil/details/48b8d670-be69-11ed-bcc6-6dacb38b2616?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/chertanovskij/services/cases/civil/details/90d28981-c281-11ed-b232-6fb0d5e1e88d?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/6afd9811-934e-11ed-aa38-dd42c3d462a2?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/tverskoj/services/cases/kas/details/9d1647c1-890c-11ee-bde3-e15247667f25?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/5b6e1581-4017-11ed-a880-fd90264a79ff?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/6d333e60-3d7b-11ed-9016-67ae9e523f4f?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/dorogomilovskij/services/cases/civil/details/9456a0d0-3750-11ed-864d-2fc2ddf82d94?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/gagarinskij/services/cases/kas/details/04cf8881-e062-11ed-8599-75e501d02ab0?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/lyublinskij/services/cases/kas/details/10e27880-0628-11ee-a80b-5135c3097443?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/kas/details/9f661f60-ab8a-11ed-9aa9-4bcb8991dd45?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/tushinskij/services/cases/criminal/details/928b9240-cf05-11ed-ae9b-fdb62f39c802?year=2023&formType=fullForm',
 'https://mos-gorsud.ru/rs/zamoskvoreckij/services/cases/criminal/details/267633c0-45b2-11ee-95af-8bf81380d527?year=2023&formType=fullForm']

In [ ]:
def get_cases_data(links):
    cases = [] # создаем пустой список
    # проходимся по всем ссылкам
    for link in tqdm.tqdm(links): 
        one_case_dict = get_info_about_case(link) # получаем информацию об одном деле в виде словаря
        cases.append(one_case_dict) # добавляем словарь в список cases
        
    # теперь используем метод .json_normalize() и получаем pandas.DataFrame
    data = pd.json_normalize([one_case for one_case in cases])
    return data

In [170]:
data # Так должен выглядеть результат

,Уникальный идентификатор дела,Номер дела ~ материала,Стороны,Дата поступления,Дата рассмотрения дела в первой инстанции,Дата вступления решения в силу,Cудья,Категория дела,Номер дела в суде вышестоящей инстанции,Текущее состояние,Решение апелляции,link,Подсудимый
0,77RS0018-02-2023-004059-43,02-4668/2023 ...,"Истец: Борсов А.И.Ответчик: ФГАОУ ВО ""Национал...",24.03.2023,20.06.2023,14.12.2023,Юдина И.В.,179 - О защите прав потребителей - из договоро...,33-48031/2023,"Удовлетворено, 20.06.2023","Отменено, 14.12.2023",https://mos-gorsud.ru/rs/nikulinskij/services/...,NaN
1,77RS0031-02-2023-003539-41,02-4564/2023 ...,Истец: Дружинин А.А.Ответчик: ФГАОУ высшего об...,03.03.2023,11.05.2023,NaN,Наделяева Е.И.,44 - О восстановлении на государственной (муни...,33-4121/2024,"Удовлетворено, 11.05.2023",NaN,https://mos-gorsud.ru/rs/horoshevskij/services...,NaN
2,77RS0002-02-2022-020618-86,02-3282/2023 ...,"Истец: Черничкин А.С.Ответчик: ПАО ""Сбербанк Р...",12.10.2022,26.07.2023,NaN,Курносова О.А.,178 - О защите прав потребителей - из договоро...,NaN,Назначено судебное заседание на 14.03.2024 09:50,NaN,https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
3,77RS0033-02-2023-003981-64,02-2500/2023 ...,Истец: Солдатенко В.А.Ответчик: Федеральное Го...,03.03.2023,15.08.2023,NaN,Молодцова Е.В.,211- Прочие исковые дела,"33-40411/2023 , ...","Удовлетворено частично, 15.08.2023",NaN,https://mos-gorsud.ru/rs/chertanovskij/service...,NaN
4,77RS0002-02-2022-023218-46,02-1845/2023 ...,Истец: Андриянова Д.А.Ответчик: Федеральное го...,25.11.2022,10.04.2023,18.05.2023,Старовойтова К.Ю.,54 - Трудовые споры о взыскании невыплаченной ...,NaN,"Удовлетворено частично, 10.04.2023",NaN,https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
5,77RS0002-02-2022-016892-12,02-1124/2023 (02-55...,"Истец: Барабанщиков К.Ю.Ответчик: ФГАОУВО ""НИУ...",17.08.2022,07.02.2023,06.09.2023,Курносова О.А.,211- Прочие исковые дела,33-38950/2023,"Отказано в удовлетворении, 07.02.2023","Отменено частично, 06.09.2023",https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
6,77RS0002-02-2022-018700-20,02-0986/2023 (02-54...,"Истец: Никонова Ю.А.Ответчик: ФГАОУ ВО ""НИУ ""В...",14.09.2022,14.02.2023,28.04.2023,Курносова О.А.,"74 - Иные, возникающие из трудовых правоотношений",NaN,"Отказано в удовлетворении, 14.02.2023",NaN,https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
7,77RS0006-02-2022-010324-07,02-0681/2023 (02-44...,Истец: Жарова А.К.Ответчик: НИУ ВШЭ,11.08.2022,15.03.2023,16.05.2023,Александренко И.М.,54 - Трудовые споры о взыскании невыплаченной ...,NaN,"Отказано в удовлетворении, 15.03.2023",NaN,https://mos-gorsud.ru/rs/dorogomilovskij/servi...,NaN
8,77RS0004-02-2023-005000-96,02а-0622/2023 ...,"Административный истец: АО ""Московский областн...",18.04.2023,21.06.2023,NaN,Кочнева А.Н.,"27 - Прочие об оспаривании решений, действий (...",NaN,"Отказано в удовлетворении, 21.06.2023",NaN,https://mos-gorsud.ru/rs/gagarinskij/services/...,NaN
9,77RS0015-02-2023-009457-13,02а-0621/2023 ...,Административный истец: Журкин М.Д.Администрат...,06.06.2023,22.06.2023,NaN,Кац Ю.А.,"26 - Об оспаривании решений, действий (бездейс...",33а-2648/2024,"Удовлетворено, 22.06.2023",NaN,https://mos-gorsud.ru/rs/lyublinskij/services/...,NaN


#### Почему метод выше – оптимален? Пояснение с примером:

**Как работает `pd.json_normalize()`?**

Эта функция принимает на вход словарь/объект json и превращает **ключи в названия колонок**, а **соответствующие им значения помещает в эти самые колонки** (см. пример выше):

```pd.json_normalize(my_dict)```

**Как может выглядеть альтернатива `pd.json_normalize()`?**

Вы могли бы обратиться к странице и не собирать значения **слева** (названия колонок) и значения **справа** (соответствующие им значения) в **словарь**, а пройтись по html разметке и попытаться сделать следующее:
```
try:
    judge = soup.find_all(..., {.. : ..})[0]
except:
    judge = np.nan
try:
    court = soup.find_all(..., {.. : ..})[0]
except:
    court = np.nan
try:
    result = soup.find_all(..., {.. : ..})[0]
except:
    result = np.nan
# etc (другие атрибуты)

df = pd.DataFrame({'judge': judge, 'court': court, 'result': result})
```


**Если не использовать `pd.json_normalize()`, вы могли бы столкнуться со следующей ситуацией** – у вас в датафрейме есть как гражданские дела, так и уголовные. Например, для дел с индексом `11`, `13`, `14` не указаны `Стороны`, зато указаны `Подсудимые` – это связано с тем что эти дела рассматривались в рамках уголовного процесса. **Если вы собираете данные с помощью функции и проверяете наличие определенных столбцов, вы рискуете собрать не все данные, потому что изначально не сможете определить а какие вообще столбцы у вас есть**.

Когда я начинала работать с судебными решениями по делам по статье 20.2 КоАП РФ, я думала что увижу 6-7 столбцов с информацией об административном деле, как в [примере](https://mos-gorsud.ru/mgs/services/cases/review-not-yet/details/a780d097-9af9-42b8-b11f-f147805ae0b6?codex=20.2%2C+Ч.+5&year=2019&formType=fullForm):

* `Уникальный идентификатор`;
* `Номер дела`;
* `Стороны`;
* `Дата регистрации`;
* `Дата окончания`;
* `Судья`;
* `Статья КоАП РФ`;
* `Результат рассмотрения`;
* `Номер дела в суде нижестоящей инстанции`.

Как вы можете заметить, как минимум последний пункт встретится явно не во всех делах – потому что если дело уже рассматривалось в суде нижестоящей инстанции, это означает, что это уже вторая инстанция рассмотрения дела; не все дела уходят во вторую инстанцию.


**Я собирала информацию со страниц дел с помощью функции, похожей на функцию выше (то есть, все-все колонки и соответствующие значения с помощью `pd.json_normalize()`) и после прохождения по всей генеральной совокупности я увидела следующие колонки:**
* `Уникальный идентификатор дела`;
* `Номер производства`;
* `Лицо, привлеченное к ответственности`;
* `Дата регистрации`;
* `Дата окончания`;
* `Дата обжалуемого решения`;
* `Cудья`;
* `Статья КоАП РФ`;
* `Текущее состояние`;
* `Номер дела`;
* `Привлекаемое лицо`;
* `Дата рассмотрения дела в первой инстанции`;
* `Результат рассмотрения`;
* `Номер дела в суде нижестоящей инстанции`;
* `Дата поступления дела в апелляционную инстанцию`;
* `Номер дела в суде вышестоящей инстанции`;
* `Другие участники`;
* `Основание решения суда`;
* `Суд, вынесший решение`;
* `Решение кассации`

Конечно, не во всех делах были все колонки (например, в итоговом датафрейме для дел в первой инстанции отсутствовали значения связанные с рассмотрением дела в суде вышестоящей инстанции – вместо них были `NaN`). **Как вы можете увидеть, этих колонок оказалось гораздо больше, чем я предполагала. Итого если у вас есть возможность получить словарь с названиями колонок и соответствующими им значениями, есть две причины использовать `pd.json_normalize()` а не функцию с `try-except` внутри:**

1. **Причина 1 – функциональная**. 
* Для того чтобы создать `pandas.DataFrame` с помощью `pd.json_normalize()`, вам нужно использовать одну строчку кода. 
* Чтобы создать `pandas.DataFrame` как в моем примере выше, вам бы пришлось 20 раз прописать `try-except`. Одна конструкция `try-except` это уже 4 строчки кода, итого получаем альтернативу в виде кода минимум на 80 строк. 

2. **Причина 2 – содержательная**. Когда вы начинаете работу с веб-страницами, вы не всегда можете сказать **какие точно значения там могут быть**. Например, страницы дел последних лет и страницы дел пять лет назад отличаются; схожие по смыслу значения там названы по разному. Кроме того, некоторые колонки, на сбор которых можно забить, вообще-то могут быть вам полезны. Например, благодаря колонке `Другие участники` я смогла увидеть, что в моей выборке есть несколько десятков дел с несовершеннолетними – потому что в этой колонке были указаны отделы ПДН; это помогло мне отфильтровать дела с несовершеннолетними обвиняемыми и рассматривать только дела с совершеннолетними и в контексте моего исследования это было важно, потому что нормативное регулирование по статье 20.2 КоАП РФ для совершеннолетних и несовершеннолетних отличается.

**Вывод – если вы можете собрать словари/json'ы с информацией и превратить их в `pandas.DataFrame`, используйте `pd.json_normalize()` и не используйте много конструкций try-except**.

## 4. Selenium: явные и неявные ожидания и обработка исключений<a name='part4'></a>

Как вы могли заметить, некоторые сайты загружаются очень постепенно. Сейчас мы с вами использовали ожидания с помощью `time.sleep(seconds)` чтобы перестраховаться от получения `ElementNotVisibleException` – исключения, которое возникает, когда вы пытаетесь взаимодействовать с элементом который еще не подгрузился на страницу. Как считают авторы документации к Selenium на Python:

> *Худший пример такого кода (явного ожидания) — это использование команды `time.sleep()`, которая устанавливает точное время ожидания.*
---

**Почему плохо использовать `time.sleep()`?** Если сайт, с которым вы работаете, долго грузится, то вы скорее всего для всех страниц будете выставлять достаточно большое значение - например, будете ждать 3-5 секунд. Однако, некоторые страницы могут загрузиться у вас быстрее (например, у вас вырастет скорость интернета). Это не страшно когда вы работаете с учебными проектами, но важно, когда занимаетесь сбором данных по работе (в том числе, от вас ждут данные заказчики или коллеги):

Представим себе следующую ситуацию:
* Вам нужно собрать данные с `10 000` страниц;
* Вы проставили 5 секунд ожидания в `time.sleep()` потому что у вас не подгружались моментально все элементы;
* Первая половина страниц действительно прогружалась по 5 секунд, **а вот для второй половины – все искомые элементы загружались за 3.5 секунды**.

**Тогда:** 
* **Вы затратили на сбор данных** с 10 тысяч страниц минимум 50 тысяч секунд (833 минуты или **13 часов 52 минуты** – собирали все десять тысяч страниц по 5 секунд), 
* **А могли бы затратить** меньше –  42.5 тысячи секунд (708 минут или **11 часов 48 минут** - собирали пять тысяч страниц по 5 секунд, другие пять тысяч страниц по 3.5 секунды). 

**Кажется, что разница небольшая – всего лишь 2 часа и 5 минут**. Но на самом деле это означает, что вам не придется писать заказчику или коллеге *Привет, данные будут через два часа* или что у вас будет время их посмотреть, написать по ним справку, оформить и дообработать при необходимости. Или вы хотя бы увидите фатальные ошибки на час раньше :)

---

Но есть еще несколько более цивилизованных способов работы с ожиданиями: **неявные** и **явные** ожидания в Selenium.

### `wb.implicitly_wait()` и неявные ожидания

`Implicit waits` – это когда вы говорите Python:
>*Ищи все элементы в течение некоторого времени*
* Если элемент не будет найден, вернется исключение `NoSuchElementException`;
* Если элемент будет найден (что может произойти и раньше чем через 10 секунд), то исполнение кода продолжится.


Другими словами, `wb.implicitly_wait(3)` позволит вам получить ошибку не сразу, а попытаться найти элементы в течение нескольких секунд. В течение этого времени `webdriver` попытается найти эти самые элементы. 

**Это глобальный таймер**, который применяется ко всем операциям поиска элементов, именно поэтому вы пишите в коде (а не используете этот код для поиска конкретных элементов):

`wb.implicitly_wait(3)`

Давайте посмотрим на примере:

In [313]:
link_to_one_case = 'https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm'

wb.implicitly_wait(3) # будем ждать 3 секунды появление любого элемента
wb.get(link_to_one_case) # открываем страницу

open_documents = wb.find_element(By.ID, 'ui-id-3') # ищем кнопку "Документы"
open_documents.click() # кликаем

### `try-except` и работа с исключениями в Selenium
Код выше был выполнен успешно, потому что у нас с вами действительно есть этот элемент на странице. А теперь давайте попробуем переписать наш код под взаимодействие с исключениями. В больших проектах важно отлавливать разные исключения – потому что они возникают по разным причинам. Вы можете адаптировать свой код под работу с разными исключениями с помощью `try-except`.

Названия всех распространенных исключений в Selenium можно найти по [ссылке](https://www.selenium.dev/selenium/docs/api/py/common/selenium.common.exceptions.html).

Рассмотрим на примере:

In [190]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import TimeoutException

link_to_one_case = 'https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm'

wb.implicitly_wait(5) # будем ждать 5 секунд нахождение ЛЮБОГО элемента
wb.get(link_to_one_case) # открываем страницу

non_existed_id = 'myDinamicElement' # придумала название элемента

# пытаемся
try:
    open_documents = wb.find_element(By.ID, non_existed_id) # ищем кнопку 
    open_documents.click() # кликаем
# Если мы получили в ходе работы блока кода выше ElementNotVisibleException (элемент не видим)
except ElementNotVisibleException: 
    print('Вебдрайвер не может увидеть этот элемент') # печатаем такую строчку
# Если мы получили в ходе работы блока кода выше NoSuchElementException (нет такого элемент)
except NoSuchElementException: 
    print('Такой элемент не найден')# печатаем такую строчку
# Если мы получили TimeoutException (время вышло)
except TimeoutException: 
    print('Время вышло!') # печатаем такую строчку

Такой элемент не найден


### Другие методы `WebElement`: `is_displayed()`, `is_enabled()`, `.is_selected()`.

Методы выше возвращают булевы значения `True`/`False`:

* `.is_displayed()` – проверяет, отображается ли элемент на странице;
* `.is_enabled()` – проверяет, доступен ли элемент для взаимодействия;
* `.is_selected()` – проверяет, выбран ли элемент (например, chechbox).

In [191]:
link_to_one_case = 'https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm'

wb.get(link_to_one_case) # открываем страницу

In [196]:
print('С открытой страницы можно скачать файлов: ', end='')
print(len(wb.find_elements(By.LINK_TEXT, 'Скачать файл')))

С открытой страницы можно скачать файлов: 1


In [197]:
wb.find_element(By.LINK_TEXT, 'Скачать файл').is_displayed() # отображается? Да

True

In [198]:
wb.find_element(By.LINK_TEXT, 'Скачать файл').is_enabled() # доступен для взаимодействия? Да

True

In [199]:
wb.find_element(By.LINK_TEXT, 'Скачать файл').is_selected() # выбран? Нет

False

В некотором смысле, могут являться альтернативой явных ожиданий или помогут для самых простых действий. Давайте продемонстрируем:

In [200]:
import time # модуль для работы с временем

start = time.time() # замеряем время старта
wb.get('https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm')
point_1 = time.time() # после того как открылась страница, вновь замеряем время

# используем чудесный цикл while 

while True: # пока не остановлен цикл,
    # если мы находим элемент по id и этот элемент доступен для взаимодействия,
    if wb.find_element(By.ID, 'ui-id-3').is_enabled():
        print('Кнопка "Документы" стала доступна.') # печатаем строчку
        end = time.time() # замеряем время конца
        break # завершаем цикл
        
print(f'Открытие страницы заняло {point_1 - start} секунд. Чтобы кнопка "Документы" стала доступна, потребовалось\
 еще {end - point_1} секунд.') # печатаем строчку

Кнопка "Документы" стала доступна.
Открытие страницы заняло 37.07115316390991 секунд. Чтобы кнопка "Документы" стала доступна, потребовалосьеще 0.014458894729614258 секунд.


Как вы видите, основное время ушло на открытие страницы. Но на появление кнопки "Документы" также ушло время. А вот сэкономить это самое время вам помогут явные ожидания.

### `WebDriverWait() и явные ожидания`

Явное ожидание (explicit waits) — **это код, которым вы определяете какое необходимое условие должно произойти для того, чтобы дальнейший код исполнился**. Другими словами, вы говорите Python:

>*Ищи данный объект в течение N секунд, и чтобы выполнялось условие (он есть на странице/есть и видим на странице/есть, видим и кликабелен на странице/etc)*

* Вы настраиваете явные ожидания для каждого конкретного элемента (а не в целом для поиска всех элементов, как в случае имплицитных ожиданий);
* Если элемент не будет найден за определенное число секунд, вернется исключение;
* Если элемент будет найден и будет удовлетворять определенному условию (есть на странице/есть и видим на странице/есть, видим и кликабелен на странице/etc), то будет сохранен в переменную `element`.

**Пример явного ожидания:**

```element = WebDriverWait(wb, 10).until(EC.element_to_be_clickable((By.ID, "ui-id-3")))```

1. Мы используем класс `WebDriverWait`, внутри обращаемся к объекту `webdriver` и говорим ему: мы будем пытаться получить этот объект в течение 10 секунд: 

```WebDriverWait(browser, 10).until()```

* Если мы найдем элемент до этого момента, то в переменную `element` будет сохранен просто элемент. 
* Если мы не найдем элемент за `10` секунд, то получим исключение. 
* Внутри `WebDriverWait` у нас есть аргумент `poll_frequency` - это частота, с которой вебдрайвер будет пытаться найти этот элемент на странице – по умолчанию это `0.5` секунд. То есть, за 10 секунд вебдрайвер попытается 20 раз найти элемент на странице.
* `.until()` – это метод, который указывает, какое условие мы ожидаем выполнить.

2. Во второй части кода указано, как мы ищем элемент:
* Мы обращаемся к `expected_conditions`;
* Выбираем класс – `element_to_be_clickable` – то есть, мы ищем не просто элемент (по атрибуту его `ID` в html разметке, `ui-id-3`), а ищем элемент и чтобы он еще был кликабельным. 

```(EC.element_to_be_clickable((By.ID, "ui-id-3")))```




**Список всех явных ожиданий доступен в [документации](https://selenium-python.readthedocs.io/waits.html)**, но приведу несколько примеров:

* `presence_of_element_located` - ожидание, которое проверяет присутствует ли элемент в html страницы (не обязательно должен быть видимым);
* `visibility_of_element_located` - ожидание, которое проверяет присутствует ли элемент в html страницы и видим ли он. Видимость означает что элемент не только отображается, но также имеет высоту (height) и ширину (width) больше, чем 0. 
* `element_to_be_clickable` – ожидание, которое проверяет, отображается и кликабелен ли элемент.

**Какие элементы мы тут используем?**

```element = wait.until(EC.element_to_be_clickable((By.ID, 'ui-id-3')))```

* `WebDriverWait` – класс, отвечающий за ожидание;
* `wait.until()` это метод, отвечающий за ожидание. Внутрь ему мы должны подать то, что и как мы ожидаем.
* `EC.<some value>` – класс, отвечащий за то, что конкретно мы ожидаем. В качестве аргумента подаем локатор элемента `(By.ID, 'ui-id-3')`.

Воспользуемся явными ожиданиями сами:

In [195]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC # загружаем явные ожидания -  класс
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

link_to_one_case = 'https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm'
wb.get(link_to_one_case) # открываем ссылку

# создаем объект wait
wait = WebDriverWait(wb, 3) # мы будем ждать до 3х секунд ИЛИ пока не найдем элемент, удовлетворяющий условиям:

try:
    # воспользуемся ожиданием, проверяющим, кликабельный ли элемент
    # В течение трех секунд пытаемся найти элемент по его ID (ui-id-3) и при этом чтобы на него можно было кликнуть
    element = wait.until(EC.element_to_be_clickable((By.ID, 'ui-id-3'))) # пытаемся нажать на кнопку "Документы"
    element.click() # кликнем
    print('Получилось')
# если получили исключение NoSuchElementException (нет такого элемента)
except NoSuchElementException:
    print('Не получилось найти элемент.')
# если получили исключение ElementNotInteractableException (с элементом нельзя провзаимодействовать)
except ElementNotInteractableException:
    print('С этим элементом нельзя взаимодействовать')

<p></p>
<center><b><font size=4>Задача №3. Явные ожидания </font></b></center>

Откройте с помощью `webdriver` страницу ВКР ОП "Политология" и выберите [ВКР с первой страницы](https://www.hse.ru/ba/political/students/diplomas) (можно воспользоваться ссылкой в переменной `link`).

In [172]:
link = 'https://www.hse.ru/ba/political/students/diplomas/833280930'

**Шаг 1. Попытайтесь получить аннотацию ВКР, отправив просто запрос с помощью `requests`**

In [314]:
# YOUR CODE HERE
html = requests.get(link)
soup = BeautifulSoup(html.text)

In [316]:
soup.find_all('div', {'class':"vkr-content__text"}) # элементы подгружаются постепенно
# и текст аннотации подгружается не сразу, поэтому мы зачастую и используем Selenium

[]

**Шаг 2. Откройте ту же страницу с помощью `webdriver`, используйте неявное ожидание с Selenium на 10 секунд и напечатайте аннотацию ВКР**

In [368]:
# YOUR CODE HERE

wb.get('https://www.hse.ru/ba/political/students/diplomas/833280930') # открываем эту работу
wb.implicitly_wait(10) # неявное ожидание - мы будем ждать появления всех элементов в дальнейшем до 10 секунд
annotation = wb.find_element(By.CLASS_NAME, "vkr-content__text") 
print(annotation.text)

Проведение российскими войсками специальной военной операции на территории Украины вызывает неоднозначную реакцию в российском обществе. Одной из причин, по которой граждане негативно оценивают политику России на Украине, являются санкции западных стран, направленные на подрыв российской экономики и создание угрозы внутренней стабильности и безопасности государства. Целью данной работы является исследование процесса легитимизации государственной политики по адаптации российской экономики к санкциям после февраля 2022 г.. В работе проводится фрейм-анализ материалов с официального сайта Минэкономразвития РФ, опубликованных в период с 24 февраля 2022 г. по 31 декабря 2022 г., и выделяются способы достижения государством одобрения населением собственной политики по адаптации российской экономики к санкциям. В первой части работы представлен обзор литературы по теме исследования, а также теоретическая рамка исследования. Во второй части приведены результаты фрейм-анализа изученных материало

**Шаг 3. Используйте явное ожидание на наличие объекта аннотации на странице и напечатайте аннотацию ВКР**

In [369]:
# YOUR CODE HERE
wb.get('https://www.hse.ru/ba/political/students/diplomas/833280930')

wait = WebDriverWait(wb, 3) # ждем до 3х секунд ИЛИ пока не найдем элемент, удовлетворяющий условиям:

try:
    # воспользуемся ожиданием, проверяющим, присутствует ли элемент,
    # ЕСЛИ элемент есть и он присутствует (.presence_of_element_located), то напечатаем текст
    # ЕСЛИ это условие не удовлетворяется, то будет вызвано исключение
    element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'vkr-content__text'))) 
    print(element.text) # и печатаем текст
except:
    print('Не получилось')

Проведение российскими войсками специальной военной операции на территории Украины вызывает неоднозначную реакцию в российском обществе. Одной из причин, по которой граждане негативно оценивают политику России на Украине, являются санкции западных стран, направленные на подрыв российской экономики и создание угрозы внутренней стабильности и безопасности государства. Целью данной работы является исследование процесса легитимизации государственной политики по адаптации российской экономики к санкциям после февраля 2022 г.. В работе проводится фрейм-анализ материалов с официального сайта Минэкономразвития РФ, опубликованных в период с 24 февраля 2022 г. по 31 декабря 2022 г., и выделяются способы достижения государством одобрения населением собственной политики по адаптации российской экономики к санкциям. В первой части работы представлен обзор литературы по теме исследования, а также теоретическая рамка исследования. Во второй части приведены результаты фрейм-анализа изученных материало

## 5. Selenium: ActionChains <a name='part5'></a>

Что такое `ActionChains`? Это способ автоматизации низкоуровневых взаимодействий, таких как движения мыши, действия с кнопками мыши, нажатие клавиш и взаимодействие с контекстным меню. 

**В чем преимущества `ActionChains`?**
* **Вы реализуете сразу цепочку действий**: с помощью ActionChains вы не просто последовательно запускаете отдельные методы, а реализуете цепочки действий, которые выполняются последовательно. Это позволяет имитировать сложные действия пользователя.


*  **Вы можете создавать сложные сценарии взаимодействия и работать с элементами, где требуется высокая точность**  – например, `ActionChains` подходит для сценариев вроде рисования на холсте или игры в браузере.

Давайте вернемся к странице **"Поиск"** и перепишем наш код с помощью `ActionChains`:

In [211]:
from selenium.webdriver.common.action_chains import ActionChains

# Открываем страницу
wb.get('https://mos-gorsud.ru/search')

expand_search_button = wb.find_element(By.CLASS_NAME, 'extended ') # находим кнопку "расширенный поиск"
open_menu_instance_var1 = wb.find_element(By.ID, 'instance-button') # находим элемент "первая инстанция"
open_menu_instance_var2 = wb.find_element(By.CSS_SELECTOR, '#instance-button > span.ui-selectmenu-text') # второй вариант поиска
# Зачем используем try-except? Потому что значение "Первая" не появляется если не кликнуть заранее на кнопку
try:
    open_menu_instance_var1.click() 
except:
    open_menu_instance_var2.click()
first_instance = wb.find_element(By.ID, 'ui-id-9') # ищем значение "Первая "


# Использование ActionChains для выполнения последовательности действий

actions = ActionChains(wb) # создаем экземпляр класса ActionChains
actions.click(expand_search_button)  # будем кликать на кнопку "расширенный поиск"
try:
    actions.click(open_menu_instance_var1)    # будем кликать на кнопку "раскрыть опции" для выбора инстанции
except:
    actions.click(open_menu_instance_var2) # если не получится, то на второй элемент
    
actions.click(first_instance)        # далее будем кликать на первую инстанцию
и                # выполним накопленные действия

А теперь давайте полностью перепишем код решения задачи №2 из ноутбука к семинару 13-14.

<p></p>
<center><b><font size=4>Задача №4. ActionChains </font></b></center>

**Используя `ActionChains`, перепишите код из ячейки выше, добавив выбор года (`2023`) и поиск по тексту судебных решений – чтобы там присутствовало словосочетание `НИУ ВШЭ`. Итого в поиске должно быть:**

* Год – `2023`;
* Инстанция – `Первая`;
* В тексте присутствовать словосочетание `НИУ ВШЭ`.

**Покажите преподавательнице, что у вас получилось**.

**Решение**

Здесь вы можете встретить работу со списковыми включениями, которые не видели раньше. Выглядит это так:

```button_2023 = [i for i in wb.find_elements(By.CLASS_NAME, 'ui-menu-item-wrapper') if i.text.strip() == '2023'][0]```

Что тут происходит?
* Вы ищите все элементы с названием класса `ui-menu-item-wrapper`;
* Проходитесь по ним в цикле;
* Оставляете в списке только те, текст за которыми `i.text.strip()` равен значению `2023`;
* Берете только первое (и единственное) значение `[i for i in ..][0]`;
* Сохраняете в переменную `button_2023`.


**Это удобный способ искать элементы по значению, которое за ним стоит**. В Selenium есть метод поиска `wb.find_element(By.LINK_TEXT, ...)` и `wb.find_element(By.PARTIAL_LINK_TEXT, ..)`, но он позволяет работать только с объектами, за которыми стоят ссылки, а не с простыми элементами на странице. Попытайтесь использовать на открытой странице поиска ```[i.text for i in wb.find_elements(By.PARTIAL_LINK_TEXT, '2023')]``` и вы это увидите:

In [412]:
# Запустите этот код и увидите, что вам вернутся только текстовые значения, которые стоят за ссылками
# Например, вы не увидите "Удовлетворено частично, 15.08.2023"
[i.text for i in wb.find_elements(By.PARTIAL_LINK_TEXT, '2023')]

['02-4668/2023',
 '02-4564/2023',
 '02-3282/2023',
 '02-2932/2023',
 '02-2500/2023',
 '02-1845/2023',
 '02а-1544/2023',
 '02-1124/2023',
 '02-0986/2023',
 '02-0681/2023',
 '02а-0622/2023',
 '02а-0621/2023',
 '02а-0484/2023',
 '01-0476/2023',
 '01-0453/2023']

In [408]:
# Решение
from selenium.webdriver.common.action_chains import ActionChains

wb.implicitly_wait(3) # устанавливаем неявное ожидание (для всех элементов что будем искать до 3х секунд)
wb.get('https://mos-gorsud.ru/search') # открываем ссылку

expand_search_button = wb.find_element(By.CLASS_NAME, 'extended ') # находим кнопку "расширенный поиск"
expand_search_button.click() # кликаем на неё

open_menu_instance_var1 = wb.find_element(By.ID, 'instance-button') # находим элемент "первая инстанция"
open_menu_instance_var2 = wb.find_element(By.CSS_SELECTOR, '#instance-button > span.ui-selectmenu-text') # второй вариант поиска
# Зачем используем try-except? Потому что значение "Первая" не появляется если не кликнуть заранее на кнопку
# Поэтому перед использованием ActionChains получаем значение ID для кнопки "Первая инстанция",
# чтобы потом не словить ошибку (ведь изначально значения еще не подгрузились)
try:
    open_menu_instance_var1.click() 
except:
    open_menu_instance_var2.click()
first_instance = wb.find_element(By.ID, 'ui-id-9') # ищем значение "Первая "

# ПЕРВЫЙ ОБЪЕКТ ACTIONCHAINS (отвечает за выбор инстанции)
# 1. Использование ActionChains для выполнения последовательности действий со значением ИНСТАНЦИЯ
actions = ActionChains(wb) # создаем экземпляр класса ActionChains
try:
    actions.click(open_menu_instance_var1)    # будем кликать на кнопку "раскрыть опции" для выбора инстанции
except:
    actions.click(open_menu_instance_var2) # если не получится, то на второй элемент
try:
    open_menu_instance_var1.click() # кликнем на кнопку еще раз чтобы раскрылся выбор вариантов
except:
    open_menu_instance_var2.click() # кликнем на кнопку еще раз
    
actions.move_to_element(first_instance) # перенесемся к варианту "Первая" (иснанция)
actions.click(first_instance)        # кликнем на значение "Первая" (инстанция)
actions.perform() # исполним накопленные действия

# ВТОРОЙ ОБЪЕКТ ACTIONCHAINS (отвечает за выбор года, поле текст и кнопку "Найти")
# 2. Теперь кликнем на поле "Год", и сперва получим значение элемента для кнопки "2023"
open_year_var1 = wb.find_element(By.ID, 'ui-id-1-button') # ищем поле "Год" - вариант 1
open_year_var2 = wb.find_element(By.CSS_SELECTOR, '#ui-id-1-button') # ищем поле "Год" - вариант 2

try:
    open_year_var1.click() # кликаем выбор года
    time.sleep(1) # чуть-чуть ждем, да, к сожалению используя обычные паузы
    # ищем кнопку "2023"
    button_2023 = [i for i in wb.find_elements(By.CLASS_NAME, 'ui-menu-item-wrapper') if i.text.strip() == '2023'][0]
except:
    open_year_var2.click()
    time.sleep(1)
    button_2023 = [i for i in wb.find_elements(By.CLASS_NAME, 'ui-menu-item-wrapper') if i.text.strip() == '2023'][0]

actions = ActionChains(wb) # создаем экземпляр класса ActionChains
actions.move_to_element(button_2023) # двигаемся к варианту "2023"
actions.click(button_2023) # кликаем на эту кнопку
# Тут начинаем работать с полем для ввода текста
text_field = wb.find_element(By.ID, 'documentText')
actions.move_to_element(text_field)
actions.click(text_field)
actions.send_keys('НИУ ВШЭ') # посылаем это словосочетание в поле, на которое кликнули только что
actions.click(wb.find_element(By.ID, 'case-index-search-form-btn')) # кликаем на кнопку "Найти"
actions.perform() 

## 6. os: менеджмент файлов на вашем компьютере <a name='part6'></a>

Модуль `os` выступает в качестве интерфейса между Python и операционной системой, что позволяет управлять путями к файлам, создавать каталоги, получать информацию о запущенных процессах и переменных окружения и выполнять множество других полезных вещей. Документация os доступна по [ссылке](https://docs.python.org/3/library/os.html). 

Здесь мы сразу же используем и модуль `glob` - он используется для поиска файлов, удовлетворяющих определенным условиям.

In [370]:
import os 
import glob

In [371]:
your_downloads = '/Users/lika.kapustina/Downloads' # поменяйте это значение на папку со своими загрузкамми

list_of_files = glob.glob(f'{your_downloads}/*') # * означает что мы ищем файлы всех форматов.
latest_file = max(list_of_files, key=os.path.getctime) # используем в качестве ключа время
print(latest_file) # напечатает название последнего по времени файла в ваших загрузках

/Users/lika.kapustina/Downloads/Динамика_внутренней_элиты_в_диктатурах.pptx


Давайте адаптируем код выше к нашему коду и будем не только скачивать в цикле все файлы, но и сохранять информацию о том, для какого дела файл мы сохранили.

In [218]:
# wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [224]:
downloaded_files = {}

for link in tqdm.tqdm(links):
    wb.get(link) # открываем дело
    wait = WebDriverWait(wb, 3) # подождем 3 секунды ИЛИ пока кнопка "документы" не будет найдена
    
    try:
        # Сценарий 1: на странице есть кнопка "Документы" и нужно кликнуть на неё чтобы увидеть файлы
        # воспользуемся ожиданием, проверяющим, кликабельный ли элемент
        element = wait.until(EC.element_to_be_clickable((By.ID, 'ui-id-3'))) # пытаемся нажать на кнопку "Документы"
        element.click() # кликаем на документы
        html = br.page_source
        soup = BeautifulSoup(html)
        links_for_files = [f"https://mos-gorsud.ru{i.get('href')}" for i in soup.find_all('a') if 'Скачать файл' in i.text] # получаем список элементов, которые можно скачать
    except:
        # Сценарий 2: на странице нет кнопки "Документы", но документы для скачивания доступны на основной странице
        try:
            links_for_files = [f"https://mos-gorsud.ru{i.get('href')}" for i in soup.find_all('a') if 'Скачать файл' in i.text] # получаем список элементов, которые можно скачать
        # Сценарий 3: на странице не найдено
        except:
            print(f'На странице {link} не найдено файлов для скачивания.')
            continue # если не было не найдено никаких файлов, сразу переходим к работе со следующей ссылкой
    
    # Перейдем к работе со следующей ссылкой если ошибок не возникло, но нет ссылок для скачивания
    if len(links_for_files) == 0:
        continue
        
    # Этот блок кода запускается если у нас есть какие-то ссылки для скачивания 
    downloaded_files[link] = [] # создаем новый ключ (ссылка) и присваиваем значение - пустой список.
     
    # Пройдемся по всем ссылкам на скачивание на странице ОДНОГО дела
    for one_file_link in links_for_files:
        wb.get(one_file_link)
        time.sleep(2) # простите мне паузу в 2 секунды, это нужно чтобы файл скачался
        
        # теперь получим название последнего скачанного файла
        list_of_files = glob.glob(f'{your_downloads}/*') # получаем список элементов которые были в папке "Загрузки"
        latest_file = max(list_of_files, key=os.path.getctime) # получаем последний скачанный элемент
        
        downloaded_files[link].append(latest_file)

100%|███████████████████████████████████████████| 30/30 [18:11<00:00, 36.38s/it]


Теперь посмотрим на то что получилось:

In [225]:
downloaded_files

{'https://mos-gorsud.ru/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm': ['/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (30).docx',
  '/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (31).docx'],
 'https://mos-gorsud.ru/rs/horoshevskij/services/cases/civil/details/63577af0-c944-11ed-ab7e-afa5fd1f1a80?year=2023&formType=fullForm': ['/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (32).docx',
  '/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (33).docx'],
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/7da80261-cd6f-11ed-91bc-678868b51ab8?year=2023&formType=fullForm': ['/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (34).docx

<p></p>
<center><b><font size=4>Задача №5. Менеджмент файлов </font></b></center>

**Воспользуйтесь поиском по документации `os`, stackoverflow и другими ресурсами и сделайте следующее**:
* С помощью `os` создайте папку `mosgorsud_hse_data` на вашем **рабочем столе**;
* Пройдитесь по всем документам из вашего словаря и, используя функции os, **переместите эти файлы в эту папку**;
* Перезапишите в словарь новый путь к файлу;
* Напечатайте словарь.

In [ ]:
download_path = '/Users/lika.kapustina/Downloads' # путь к моей папке с загрузками
new_path = '/Users/lika.kapustina/Desktop/mosgorsud_hse_data/' # создаю путь, НО не папку(!)
# идем по словарю, по ключу и по значению
for link, files in downloaded_files.items():
    new_pathes = [] # сюда будем сохранять новые адреса файлов
    # идем по файлам (их может быть несколько)
    for file in files:
        # пробуем обработать файл
        try:
            current_name_of_file = file # нынешний абсолютный адрес файла
            new_name_of_file = current_name_of_file.replace(download_path, new_path) # создаем новый путь
            os.renames(current_name_of_file, new_name_of_file) # перемещаю файл с помощью os.renames()
            new_pathes.append(new_name_of_file)
        # если возникает ошибка, то 
        except Exception as e:
            print(e) # печатаем ошибку
            new_pathes.append(file) # вновь добавляем путь в файл
    # В конце обновляем словарь: перезаписываем туда новые пути к файлам
    downloaded_files[link] = new_pathes

## Дополнительно: что еще можно делать с помощью Selenium? Краткий обзор возможностей<a name='partlast'></a>

**Работа с Selenium предоставляет вам множество возможностей. Вот только некоторые из них:**
* Поиск элементов по локаторам (семинар 13-14);
* Автоматизированный ввод значений в поля и отправка форм (семинар 13-14);
* Клики по элементам (семинар 13-14);
* Управление браузером, включая:
    * Обновление страницы;
    * Переключение по вкладкам;
    * Возвращение на предыдущую вкладку, etc.
* Скроллинг страниц, включая:
    * Перемещение вверх и вниз страницы;
    * Перемещение к конкретному элементу;
    * Использование курсора и нажатия на элементы ПКМ и ЛКМ;
* Работа с cookies:
    * Получение текущих cookies;
    * Отправка заранее подготовленного объекта cookies;
* Запускать код javascript в браузере;
* Использование методов drag-and-drop: перемещение элементов из одной точки экрана в другую;
* Работа с всплывающими (модальными) окнами (включая предупреждения);
* Получение скриншотов экрана.

Найти как реализовать эти возможности Selenium можно в интернете, на stackoverflow или в официальной документации. Selenium предлагает вам много возможностей для автоматизации действий, включая сбор данных, и я надеюсь что то что мы с вами обсудили будет полезно для ваших учебных и рабочих проектов.